In [1]:
import pandas as pd
import numpy as np
import random

## Kai's Code

In [0]:
glob_pop = []

# states
states = ['well', 'infected-a', 'infected-s', 'recovered', 'dead']
class dude:
      def __init__(self, occupation, age, state=0, underlying_conditions=False, network=[]):
        self.job = occupation
        self.age = age
        self.state = state
        self.under_cond = underlying_conditions
        self.network = {'john': 'near', 'james':'far'}
        self.network = [groupa]
        
class group:
    def __init__(self):
        self.transmission_rate = 0.0
        self.demographics = [0, 0, 0]
        self.infected_count = 0
        self.memebers = []
        
        

Eddie = dude('developer', age=21)
Kai = dude('student', age=21)
Sihan = dude(occupation='student', age=22, state=0)



## Eddie's Code

In [27]:
glob_pop = []

# states
states = ['well', 'infected-a', 'infected-s', 'recovered', 'dead']
jobs = ['student', 'doctor', 'lawyer', 'police person', 'shop assistant', 'unemployed', 'retired']
job_type = ['student', 'high-income', 'low-income', 'retired']
class dude:
    def __init__(self, occupation, age, state=0, underlying_conditions=False, network=[]):
        self.job = occupation
        self.age = age
        self.state = state
        self.under_cond = underlying_conditions
        self.network = network
    
    def __str__(self):
        
            return "Job: {}, Age: {}, State: {}".format(self.job, self.age, self.state)
        
        
class group:
    def __init__(self, size, rate):
        self.transmission_rate = rate
        self.demographics = [0, 0, 0]
        self.infected_count = 0
        self.size = size
        self.members = []
        
    def __str__(self):
        
            return "Size: {}, Infected: {}".format(self.size, self.infected_count)

        

Initial population creation and first circle

In [28]:
comm_size = 1000

houses = []
for i in range(0, comm_size):
    rate = random.random()
    size = random.randint(1, 7)
    grp = group(size, rate)
    houses.append(grp)

In [29]:
peeps = []
for i in range(0, len(houses)):
    job = random.choice(job_type)
    age = random.randint(18, 80)
    person = dude(job, age, network=[houses[i]])
    houses[i].members.append(person)
    peeps.append(person)

In [30]:
for pep in peeps:
    print(pep)

Job: high-income, Age: 60, State: 0
Job: low-income, Age: 79, State: 0
Job: low-income, Age: 62, State: 0
Job: student, Age: 38, State: 0
Job: low-income, Age: 40, State: 0
Job: retired, Age: 78, State: 0
Job: student, Age: 78, State: 0
Job: high-income, Age: 27, State: 0
Job: low-income, Age: 66, State: 0
Job: student, Age: 49, State: 0
Job: student, Age: 76, State: 0
Job: student, Age: 60, State: 0
Job: high-income, Age: 61, State: 0
Job: retired, Age: 79, State: 0
Job: high-income, Age: 48, State: 0
Job: retired, Age: 72, State: 0
Job: high-income, Age: 33, State: 0
Job: retired, Age: 74, State: 0
Job: retired, Age: 46, State: 0
Job: low-income, Age: 21, State: 0
Job: student, Age: 51, State: 0
Job: high-income, Age: 19, State: 0
Job: high-income, Age: 53, State: 0
Job: low-income, Age: 30, State: 0
Job: student, Age: 24, State: 0
Job: student, Age: 44, State: 0
Job: low-income, Age: 59, State: 0
Job: retired, Age: 32, State: 0
Job: low-income, Age: 53, State: 0
Job: retired, Age: 5

Job: low-income, Age: 68, State: 0
Job: high-income, Age: 21, State: 0
Job: low-income, Age: 44, State: 0
Job: low-income, Age: 20, State: 0
Job: low-income, Age: 38, State: 0
Job: retired, Age: 56, State: 0
Job: high-income, Age: 59, State: 0
Job: student, Age: 31, State: 0
Job: student, Age: 61, State: 0
Job: high-income, Age: 38, State: 0
Job: high-income, Age: 41, State: 0
Job: high-income, Age: 33, State: 0
Job: low-income, Age: 45, State: 0
Job: low-income, Age: 62, State: 0
Job: high-income, Age: 67, State: 0
Job: high-income, Age: 18, State: 0
Job: retired, Age: 47, State: 0
Job: student, Age: 40, State: 0
Job: student, Age: 59, State: 0
Job: student, Age: 27, State: 0
Job: low-income, Age: 51, State: 0
Job: high-income, Age: 59, State: 0
Job: student, Age: 34, State: 0
Job: high-income, Age: 23, State: 0
Job: student, Age: 37, State: 0
Job: retired, Age: 77, State: 0
Job: high-income, Age: 22, State: 0
Job: low-income, Age: 46, State: 0
Job: retired, Age: 19, State: 0
Job: ret

In [31]:
for house in houses:
    job = house.members[0].job
    
    for new_member in range(0, house.size -1):
        age = abs(house.members[0].age + random.randint(-5, 5))
        person = dude(job, age, network=[house])
        house.members.append(person)
        peeps.append(person)
        

In [11]:
for house in houses:
    print(house)
    for i, mem in enumerate(house.members):
        print(i, mem)

Size: 7, Infected: 0
0 Job: high-income, Age: 59, State: 0
1 Job: high-income, Age: 60, State: 0
2 Job: high-income, Age: 61, State: 0
3 Job: high-income, Age: 61, State: 0
4 Job: high-income, Age: 55, State: 0
5 Job: high-income, Age: 59, State: 0
6 Job: high-income, Age: 61, State: 0
Size: 6, Infected: 0
0 Job: retired, Age: 58, State: 0
1 Job: retired, Age: 55, State: 0
2 Job: retired, Age: 63, State: 0
3 Job: retired, Age: 55, State: 0
4 Job: retired, Age: 54, State: 0
5 Job: retired, Age: 61, State: 0
Size: 7, Infected: 0
0 Job: student, Age: 37, State: 0
1 Job: student, Age: 34, State: 0
2 Job: student, Age: 33, State: 0
3 Job: student, Age: 41, State: 0
4 Job: student, Age: 40, State: 0
5 Job: student, Age: 40, State: 0
6 Job: student, Age: 41, State: 0
Size: 1, Infected: 0
0 Job: high-income, Age: 55, State: 0
Size: 2, Infected: 0
0 Job: high-income, Age: 28, State: 0
1 Job: high-income, Age: 31, State: 0
Size: 7, Infected: 0
0 Job: student, Age: 52, State: 0
1 Job: student, Ag

1 Job: student, Age: 63, State: 0
2 Job: student, Age: 63, State: 0
3 Job: student, Age: 66, State: 0
4 Job: student, Age: 62, State: 0
5 Job: student, Age: 67, State: 0
Size: 3, Infected: 0
0 Job: low-income, Age: 64, State: 0
1 Job: low-income, Age: 59, State: 0
2 Job: low-income, Age: 66, State: 0
Size: 3, Infected: 0
0 Job: low-income, Age: 30, State: 0
1 Job: low-income, Age: 33, State: 0
2 Job: low-income, Age: 33, State: 0
Size: 6, Infected: 0
0 Job: student, Age: 61, State: 0
1 Job: student, Age: 57, State: 0
2 Job: student, Age: 59, State: 0
3 Job: student, Age: 58, State: 0
4 Job: student, Age: 56, State: 0
5 Job: student, Age: 61, State: 0
Size: 6, Infected: 0
0 Job: retired, Age: 74, State: 0
1 Job: retired, Age: 79, State: 0
2 Job: retired, Age: 75, State: 0
3 Job: retired, Age: 73, State: 0
4 Job: retired, Age: 70, State: 0
5 Job: retired, Age: 69, State: 0
Size: 7, Infected: 0
0 Job: low-income, Age: 53, State: 0
1 Job: low-income, Age: 58, State: 0
2 Job: low-income, Ag

Size: 2, Infected: 0
0 Job: high-income, Age: 44, State: 0
1 Job: high-income, Age: 46, State: 0
Size: 7, Infected: 0
0 Job: low-income, Age: 59, State: 0
1 Job: low-income, Age: 54, State: 0
2 Job: low-income, Age: 58, State: 0
3 Job: low-income, Age: 54, State: 0
4 Job: low-income, Age: 64, State: 0
5 Job: low-income, Age: 61, State: 0
6 Job: low-income, Age: 60, State: 0
Size: 6, Infected: 0
0 Job: low-income, Age: 35, State: 0
1 Job: low-income, Age: 33, State: 0
2 Job: low-income, Age: 36, State: 0
3 Job: low-income, Age: 33, State: 0
4 Job: low-income, Age: 36, State: 0
5 Job: low-income, Age: 34, State: 0
Size: 7, Infected: 0
0 Job: low-income, Age: 38, State: 0
1 Job: low-income, Age: 38, State: 0
2 Job: low-income, Age: 38, State: 0
3 Job: low-income, Age: 41, State: 0
4 Job: low-income, Age: 36, State: 0
5 Job: low-income, Age: 38, State: 0
6 Job: low-income, Age: 37, State: 0
Size: 5, Infected: 0
0 Job: low-income, Age: 80, State: 0
1 Job: low-income, Age: 82, State: 0
2 Job

3 Job: retired, Age: 70, State: 0
Size: 6, Infected: 0
0 Job: high-income, Age: 65, State: 0
1 Job: high-income, Age: 70, State: 0
2 Job: high-income, Age: 68, State: 0
3 Job: high-income, Age: 66, State: 0
4 Job: high-income, Age: 64, State: 0
5 Job: high-income, Age: 63, State: 0
Size: 3, Infected: 0
0 Job: student, Age: 52, State: 0
1 Job: student, Age: 57, State: 0
2 Job: student, Age: 56, State: 0
Size: 5, Infected: 0
0 Job: high-income, Age: 18, State: 0
1 Job: high-income, Age: 13, State: 0
2 Job: high-income, Age: 15, State: 0
3 Job: high-income, Age: 15, State: 0
4 Job: high-income, Age: 22, State: 0
Size: 5, Infected: 0
0 Job: high-income, Age: 26, State: 0
1 Job: high-income, Age: 21, State: 0
2 Job: high-income, Age: 28, State: 0
3 Job: high-income, Age: 27, State: 0
4 Job: high-income, Age: 26, State: 0
Size: 1, Infected: 0
0 Job: high-income, Age: 68, State: 0
Size: 3, Infected: 0
0 Job: retired, Age: 80, State: 0
1 Job: retired, Age: 84, State: 0
2 Job: retired, Age: 84,

0 Job: student, Age: 79, State: 0
1 Job: student, Age: 80, State: 0
2 Job: student, Age: 79, State: 0
3 Job: student, Age: 82, State: 0
Size: 1, Infected: 0
0 Job: high-income, Age: 33, State: 0
Size: 7, Infected: 0
0 Job: high-income, Age: 44, State: 0
1 Job: high-income, Age: 45, State: 0
2 Job: high-income, Age: 46, State: 0
3 Job: high-income, Age: 44, State: 0
4 Job: high-income, Age: 45, State: 0
5 Job: high-income, Age: 47, State: 0
6 Job: high-income, Age: 46, State: 0
Size: 2, Infected: 0
0 Job: retired, Age: 52, State: 0
1 Job: retired, Age: 53, State: 0
Size: 7, Infected: 0
0 Job: retired, Age: 59, State: 0
1 Job: retired, Age: 61, State: 0
2 Job: retired, Age: 60, State: 0
3 Job: retired, Age: 55, State: 0
4 Job: retired, Age: 60, State: 0
5 Job: retired, Age: 58, State: 0
6 Job: retired, Age: 62, State: 0
Size: 5, Infected: 0
0 Job: low-income, Age: 33, State: 0
1 Job: low-income, Age: 36, State: 0
2 Job: low-income, Age: 32, State: 0
3 Job: low-income, Age: 31, State: 0
4

...

0 Job: low-income, Age: 44, State: 0
1 Job: low-income, Age: 44, State: 0
2 Job: low-income, Age: 44, State: 0
3 Job: low-income, Age: 41, State: 0
4 Job: low-income, Age: 42, State: 0
Size: 4, Infected: 0
0 Job: student, Age: 28, State: 0
1 Job: student, Age: 32, State: 0
2 Job: student, Age: 27, State: 0
3 Job: student, Age: 30, State: 0
Size: 4, Infected: 0
0 Job: low-income, Age: 80, State: 0
1 Job: low-income, Age: 83, State: 0
2 Job: low-income, Age: 82, State: 0
3 Job: low-income, Age: 83, State: 0
Size: 6, Infected: 0
0 Job: high-income, Age: 24, State: 0
1 Job: high-income, Age: 26, State: 0
2 Job: high-income, Age: 24, State: 0
3 Job: high-income, Age: 21, State: 0
4 Job: high-income, Age: 27, State: 0
5 Job: high-income, Age: 26, State: 0
Size: 1, Infected: 0
0 Job: retired, Age: 71, State: 0
Size: 1, Infected: 0
0 Job: low-income, Age: 18, State: 0
Size: 7, Infected: 0
0 Job: retired, Age: 41, State: 0
1 Job: retired, Age: 46, State: 0
2 Job: retired, Age: 46, State: 0
3 Jo

3 Job: retired, Age: 80, State: 0
4 Job: retired, Age: 74, State: 0
5 Job: retired, Age: 73, State: 0
Size: 1, Infected: 0
0 Job: retired, Age: 22, State: 0
Size: 2, Infected: 0
0 Job: low-income, Age: 26, State: 0
1 Job: low-income, Age: 31, State: 0
Size: 3, Infected: 0
0 Job: retired, Age: 69, State: 0
1 Job: retired, Age: 67, State: 0
2 Job: retired, Age: 69, State: 0
Size: 4, Infected: 0
0 Job: student, Age: 18, State: 0
1 Job: student, Age: 20, State: 0
2 Job: student, Age: 14, State: 0
3 Job: student, Age: 22, State: 0
Size: 2, Infected: 0
0 Job: high-income, Age: 41, State: 0
1 Job: high-income, Age: 44, State: 0
Size: 1, Infected: 0
0 Job: high-income, Age: 55, State: 0
Size: 7, Infected: 0
0 Job: low-income, Age: 61, State: 0
1 Job: low-income, Age: 66, State: 0
2 Job: low-income, Age: 61, State: 0
3 Job: low-income, Age: 61, State: 0
4 Job: low-income, Age: 56, State: 0
5 Job: low-income, Age: 65, State: 0
6 Job: low-income, Age: 64, State: 0
Size: 1, Infected: 0
0 Job: stud

1 Job: low-income, Age: 45, State: 0
2 Job: low-income, Age: 54, State: 0
3 Job: low-income, Age: 55, State: 0
4 Job: low-income, Age: 54, State: 0
Size: 4, Infected: 0
0 Job: low-income, Age: 33, State: 0
1 Job: low-income, Age: 32, State: 0
2 Job: low-income, Age: 30, State: 0
3 Job: low-income, Age: 33, State: 0
Size: 3, Infected: 0
0 Job: student, Age: 72, State: 0
1 Job: student, Age: 71, State: 0
2 Job: student, Age: 76, State: 0
Size: 1, Infected: 0
0 Job: high-income, Age: 23, State: 0
Size: 6, Infected: 0
0 Job: low-income, Age: 29, State: 0
1 Job: low-income, Age: 34, State: 0
2 Job: low-income, Age: 31, State: 0
3 Job: low-income, Age: 30, State: 0
4 Job: low-income, Age: 27, State: 0
5 Job: low-income, Age: 32, State: 0
Size: 7, Infected: 0
0 Job: high-income, Age: 45, State: 0
1 Job: high-income, Age: 42, State: 0
2 Job: high-income, Age: 49, State: 0
3 Job: high-income, Age: 44, State: 0
4 Job: high-income, Age: 49, State: 0
5 Job: high-income, Age: 46, State: 0
6 Job: hi

2 Job: retired, Age: 38, State: 0
3 Job: retired, Age: 43, State: 0
Size: 3, Infected: 0
0 Job: retired, Age: 28, State: 0
1 Job: retired, Age: 25, State: 0
2 Job: retired, Age: 24, State: 0
Size: 7, Infected: 0
0 Job: student, Age: 28, State: 0
1 Job: student, Age: 33, State: 0
2 Job: student, Age: 32, State: 0
3 Job: student, Age: 26, State: 0
4 Job: student, Age: 26, State: 0
5 Job: student, Age: 25, State: 0
6 Job: student, Age: 24, State: 0
Size: 3, Infected: 0
0 Job: retired, Age: 61, State: 0
1 Job: retired, Age: 61, State: 0
2 Job: retired, Age: 62, State: 0
Size: 6, Infected: 0
0 Job: high-income, Age: 78, State: 0
1 Job: high-income, Age: 74, State: 0
2 Job: high-income, Age: 75, State: 0
3 Job: high-income, Age: 82, State: 0
4 Job: high-income, Age: 74, State: 0
5 Job: high-income, Age: 78, State: 0
Size: 3, Infected: 0
0 Job: low-income, Age: 77, State: 0
1 Job: low-income, Age: 79, State: 0
2 Job: low-income, Age: 78, State: 0
Size: 7, Infected: 0
0 Job: low-income, Age: 1

2 Job: high-income, Age: 35, State: 0
3 Job: high-income, Age: 34, State: 0
Size: 1, Infected: 0
0 Job: student, Age: 36, State: 0
Size: 1, Infected: 0
0 Job: high-income, Age: 67, State: 0
Size: 5, Infected: 0
0 Job: student, Age: 76, State: 0
1 Job: student, Age: 71, State: 0
2 Job: student, Age: 75, State: 0
3 Job: student, Age: 77, State: 0
4 Job: student, Age: 77, State: 0
Size: 2, Infected: 0
0 Job: retired, Age: 73, State: 0
1 Job: retired, Age: 76, State: 0
Size: 7, Infected: 0
0 Job: low-income, Age: 37, State: 0
1 Job: low-income, Age: 40, State: 0
2 Job: low-income, Age: 41, State: 0
3 Job: low-income, Age: 41, State: 0
4 Job: low-income, Age: 40, State: 0
5 Job: low-income, Age: 34, State: 0
6 Job: low-income, Age: 37, State: 0
Size: 5, Infected: 0
0 Job: retired, Age: 65, State: 0
1 Job: retired, Age: 68, State: 0
2 Job: retired, Age: 67, State: 0
3 Job: retired, Age: 62, State: 0
4 Job: retired, Age: 68, State: 0
Size: 5, Infected: 0
0 Job: high-income, Age: 23, State: 0


Second Circle

In [32]:
len(houses)

1000

In [45]:
# Size of outer circles
circle_count = int(len(peeps) * 0.008 + random.randint(-5, 5))

In [46]:
circle_count

37

In [ ]:
for i in range(0, len(houses)):
    job = random.choice(job_type)
    age = random.randint(18, 80)
    person = dude(job, age, network=[houses[i]])
    houses[i].members.append(person)
    peeps.append(person)